# PACKAGES

In [2]:
import numpy as np
from numpy import asarray
from PIL import Image
import os, random

import matplotlib.pyplot as plt
from ripser import ripser
from persim import plot_diagrams


from sklearn.metrics import confusion_matrix

# BINARIZATION TEST

In [11]:
def BinarizeImg(image_directory):

    # create the "Binarized" folder if it doesn't exist
    binarized_directory = os.path.join(image_directory, "Binarized")
    if not os.path.exists(binarized_directory):
        os.makedirs(binarized_directory)

    # loop through each file in the directory
    for i, filename in enumerate(os.listdir(image_directory)):

        # check if the file is an image and if it's the 10th image
        if (filename.endswith(".jpg")) and (i % 8 == 0) and filename.endswith("07009.jpg"):

            # open the image using PIL
            img = Image.open(os.path.join(image_directory, filename))

            # convert the image to grayscale
            img = img.convert('L')

            #BIN 128 - ARRAY MODE
            # binarize the image
            #img_array = np.array(img) #array ogni casella è un pixel
            #binary_array = np.where(img_array < 128, 0, 255) #alzare o dimunuire threshold?
            #img = Image.fromarray(binary_array.astype(np.uint8))


            # create a new image with the same size as the original image
            binary_img = Image.new('L', img.size)

            #BIN 128 - FOR MODE
            # binarize the image
            for i in range(img.size[0]):
                for j in range(img.size[1]):
                    if img.getpixel((i, j)) < 128:
                        binary_img.putpixel((i, j), 0)
                    else:
                        binary_img.putpixel((i, j), 255)


            # save the binarized image
            #img.save(os.path.join(image_directory+"/Binarized", "binarized_" + filename))
            binary_img.save(os.path.join(image_directory+"/Binarized", filename[:-4] + "_binarized.jpg"))


            #BIN 110
            # create a new image with the same size as the original image
            binary_img2 = Image.new('L', img.size)

            # binarize the image
            for i in range(img.size[0]):
                for j in range(img.size[1]):
                    if img.getpixel((i, j)) < 110:
                        binary_img2.putpixel((i, j), 0)
                    else:
                        binary_img2.putpixel((i, j), 255)

            # save the binarized image
            #img.save(os.path.join(image_directory+"/Binarized", "binarized_" + filename))
            binary_img2.save(os.path.join(image_directory+"/Binarized", filename[:-4] + "_binarized2.jpg" ))


            #BIN AVG NEIGH
            # binarize the image using neighborhood mean as threshold
            neigh=25
            for j in range(10):
                img_array = np.array(img)
                height, width = img_array.shape
                binary_array = np.zeros((height, width), dtype=np.uint8)
                padded_array = np.pad(img_array, ((neigh,neigh),(neigh, neigh)), mode='constant', constant_values=0)
                for k in range(neigh, height+neigh):
                    for l in range(neigh, width+neigh):
                        threshold = int(np.mean(padded_array[k-neigh:k+neigh, l-neigh:l+neigh]))
                        binary_array[k-neigh, l-neigh] = 0 if img_array[k-neigh, l-neigh] < threshold else 255
                # create a new PIL image from the binary array
                binary_img3= Image.fromarray(binary_array)

                # save the final binarized image
                binary_img3.save(os.path.join(binarized_directory, filename[:-4] + "_binarized3.jpg"))


            #BIN AVG
            # binarize the image
            img_array = np.array(img)
            threshold = np.mean(img_array)
            binary_array = np.where(img_array < threshold, 0, 255)

            binary_img4 = Image.fromarray(binary_array.astype(np.uint8))

            # save the final binarized image
            binary_img4.save(os.path.join(binarized_directory, filename[:-4] + "_binarized4.jpg"))

In [12]:
# set directory containing the images
positive_img_dir = "Positive"
negative_img_dir = "Negative"

#Binarize the positive and negative images
BinarizeImg(positive_img_dir)
BinarizeImg(negative_img_dir)

# PADDING

In [13]:
def PaddingImg(image_directory, padding_size):

    # create a directory for the padded images if it doesn't exist
    padded_directory = os.path.join(image_directory, "Padded")
    if not os.path.exists(padded_directory):
        os.makedirs(padded_directory)

    # loop through each file in the directory
    for filename in os.listdir(image_directory):

        # check if the file is an image
        if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):

            # open the image using PIL
            img = Image.open(os.path.join(image_directory, filename))

            # convert the image to a NumPy array
            img_array = np.array(img)

            # get the current image size
            #height, width, depth = img_array.shape
            height, width = img_array.shape

            # create a new NumPy array with the desired size and fill with 0 (black)
            #new_img_array = np.zeros((height + 2 * padding_size, width + 2 * padding_size, depth), dtype=np.uint8)
            new_img_array = np.zeros((height + 2 * padding_size, width + 2 * padding_size), dtype=np.uint8)

            # paste the original image in the center of the new image
            new_img_array[padding_size:padding_size + height, padding_size:padding_size + width] = img_array

            # convert the NumPy array back to a Pillow image object
            new_img = Image.fromarray(new_img_array)

            # save the padded image
            new_img.save(os.path.join(padded_directory, "padded_" + filename))

        else:
            print("Errore! " + filename + " non è un'immagine")

In [15]:
# set directory containing the images
positive_img_dir = "Positive/Binarized"
negative_img_dir = "Negative/Binarized"
#Padding the positive and negative images
PaddingImg(positive_img_dir, 1)
PaddingImg(negative_img_dir, 1)

Errore! Padded non è un'immagine
Errore! Padded non è un'immagine


In [ ]:
# PERSISTENCE DIAGRAM ON SINGLE IMG TEST

In [ ]:
import matplotlib.pyplot as plt

# path to the image
image_path = 'Positive/Binarized/Padded/padded_binarized_07009.jpg'

# process the image
img = Image.open(image_path)
points = []
for i in range(img.size[0]):
    for j in range(img.size[1]):
        if img.getpixel((i, j)) == 0:
            points.append((i, j))

points = np.array(points)  # Convert to numpy array
diagrams = ripser(points, coeff=2, maxdim=1)['dgms']
print(diagrams)

plot_diagrams(diagrams, show=True)
plt.legend()
plt.clf()